# The Issue Description

The company X recruits some specialists for startups. The company uses different online marketing channels to get in touch with them.

When a specialists click on an ad, they can register on the company website. After that, company X tries to find a suitable position for them in their job database. When a match is found, the specialist is introduced to a startup and goes into an application process. Sometimes specialists are introduced to multiple startups, to improve their chances of being hired. If a specialist is finally hired, the startup pays a small fee to company X for their services. For the specialists the entire process is free.

Below are three samples.

Using it we're going to find out following:
1. Most profitable marketing channel.
2. Problematic recruitment agent(s).
3. Company wide CLTV divided by CAC.
4. Approaches to determine varying levels of conservativeness for the previous answer (how optimistic or pessimistic calculation is).

In [1]:
import sqlite3 as sl
import pandas as pd
import typing as tp

In [2]:
df_leads = pd.read_csv("leads.csv", sep = ",")
df_deals = pd.read_csv("deals.csv", sep = ",")
df_marketing = pd.read_csv("marketing.csv", sep = ",")

In [3]:
def simpe_review(df: tp.Any) -> str:
    print(
    "The structure of the table orders. "
    +"\n\n"
    +"The size of the sample: " + str(df.shape[0]) + " rows"
    +"\n\n"
    +str(df.dtypes)
    )

#### Leads 

Leads represent data scientists registering on the website. The lead owner is manually assigned. The channel field represents the marketing channel the lead was generated from.

In [4]:
simpe_review(df_leads)

The structure of the table orders. 

The size of the sample: 9073 rows

lead_id             int64
lead_created_at    object
channel            object
candidate_id        int64
lead_owner_id       int64
dtype: object


#### Deals 

Deals are opportunities which arise from a lead. They are created when a specialist is introduced to a startup. When a deal is created, it is assigned to a recruitment agents, who is responsible for the entire process from introduction to contract signing. 

A deal can either be 'won', 'lost' or 'open'. Upon creation deals are assigned with the status 'open'. A successful hire of a specialist by a startup is represented by the status 'won'. When the introduction does not lead to a hire, the deal is marked as 'lost'.

In [5]:
simpe_review(df_deals)

The structure of the table orders. 

The size of the sample: 981 rows

deal_id             int64
lead_id             int64
deal_created_at    object
agent_id            int64
value               int64
status             object
dtype: object


#### Marketing 

The marketing table records the daily spend on online marketing per marketing channel.

In [6]:
simpe_review(df_marketing)

The structure of the table orders. 

The size of the sample: 59 rows

date               object
spend_bing        float64
spend_facebook    float64
spend_adwords     float64
dtype: object


# Data Preparation

Cause we're going to solve it via SQL, let's prepare the workflow.

I'm going to use SQLITE instead of PostgreSQL - sorry, it would take fewer time and I guess the difference in syntax would be minimal.

In [7]:
def insert_to_db(sql_drop: str, sql_create: str, sql_insert: str, df_ins: tp.Any, con: tp.Any, cur: tp.Any) -> str:
    """
    UDF for creating tables and inserting data using sqlite3
    """
    
    #I've noticed SQLITE usually convert int data to blob type, so let's convert it
    for i, column in enumerate(df_ins.columns):
        if df_ins[column].dtype == 'int64':
            df_ins[column] = df_ins[column].astype(object)
    
    #connection to db
    with con:
        try:
            cur.execute(sql_drop)
            cur.execute(sql_create)

            #inserting data to table
            for i in range(df_ins.shape[0]):
                data = []
                data.append(tuple(df_ins.loc[i]))            
                cur.executemany(sql_insert, data)  

        except sl.DatabaseError as err:       
            return("Error: ", err)
        else:
            con.commit()
    return("Data was successfully inserted")

In [8]:
sql_drop = "drop table if exists leads;"
sql_create = """
            create table leads
            (
                lead_id int
                , lead_created_at text
                , channel text
                , candidate_id int
                , lead_owner_id int            
            );
            """
sql_insert = "insert into leads (lead_id, lead_created_at, channel, candidate_id, lead_owner_id) values (?, ?, ?, ?, ?);"

con = sl.connect("mwdwing_base.db")
cur = con.cursor()

insert_to_db(sql_drop, sql_create, sql_insert, df_ins = df_leads, con = con, cur = cur)

'Data was successfully inserted'

In [9]:
sql_drop = "drop table if exists deals;"
sql_create = """
            create table deals
            (
                deal_id text
                , lead_id Integer
                , deal_created_at text
                , agent_id int
                , value real
                , status text            
            );
            """
sql_insert = "insert into deals (deal_id, lead_id, deal_created_at, agent_id, value, status) values (?, ?, ?, ?, ?, ?);"

con = sl.connect("mwdwing_base.db")
cur = con.cursor()

insert_to_db(sql_drop, sql_create, sql_insert, df_ins = df_deals, con = con, cur = cur)

'Data was successfully inserted'

In [10]:
sql_drop = "drop table if exists marketing;"
sql_create = """
            create table marketing
            (
                date text
                , spend_bing real
                , spend_facebook real
                , spend_adwords real
            );
            """
sql_insert = "insert into marketing (date, spend_bing, spend_facebook, spend_adwords) values (?, ?, ?, ?);"

con = sl.connect("mwdwing_base.db")
cur = con.cursor()

insert_to_db(sql_drop, sql_create, sql_insert, df_ins = df_marketing, con = con, cur = cur)

'Data was successfully inserted'

# Data Validation

### Validation point 1. 

I'm not sure how exactly we should divide leads and candidates here.  

In [11]:
sql = """
select
count(*)
, count(distinct lead_id)
, count(distinct candidate_id)
from leads
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,count(*),count(distinct lead_id),count(distinct candidate_id)
0,9073,9073,9062


Do we have such cases when the same candidate became the lead twice or even more?

In [12]:
sql = """

select 
t.* 
from 
(
    select 
    t.*
    , max(rnk) over (partition by t.candidate_id) as max_rnk
    from 
    (
        select
        lead_id
        , candidate_id
        , channel
        , lead_created_at
        , lead_owner_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads
    ) as t 
) as t
where max_rnk > 1
limit 20
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,lead_id,candidate_id,channel,lead_created_at,lead_owner_id,rnk,max_rnk
0,53797,51672,SEM_Google,2019-01-01T11:38:06.000Z,5765548,1,2
1,60969,51672,SEM_Google,2019-02-05T13:14:32.000Z,5765548,2,2
2,55433,53255,Facebook,2019-01-09T19:34:06.000Z,5523573,1,2
3,62919,53255,Facebook,2019-02-14T18:08:51.000Z,5523573,2,2
4,57531,55326,SEM_Google,2019-01-20T10:02:43.000Z,7235120,1,2
5,60327,55326,SEM_Google,2019-02-01T19:46:25.000Z,7235120,2,2
6,57789,55565,SEM_Google,2019-01-21T16:13:07.000Z,3212193,1,2
7,57790,55565,SEM_Google,2019-01-21T16:13:07.000Z,3212193,2,2
8,59377,57102,SEM_Google,2019-01-28T20:32:46.000Z,3212193,1,3
9,59378,57102,SEM_Google,2019-01-28T20:32:46.000Z,3212193,2,3


Basically we can notice that candidates in such cases have the same channels and often the same created date. So that's why below we will use candidate_id instead of lead_id - or only first created lead_id per candidate_id.

The leads below should be excluded from further calculations - it's duplicated cases, we will use only first appearence of candidates as leads.

In [13]:
sql = """
with leads_unique as
(
    select 
    lead_id
    , rnk
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk > 1
)

select * from leads_unique

"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,lead_id,rnk
0,60969,2
1,62919,2
2,60327,2
3,57790,2
4,59378,2
5,59379,3
6,61046,2
7,61048,3
8,65803,2
9,65645,2


### Validation point 2. 

I'm not sure regarding correspondence of leads creation dates and marketing dates.

In [14]:
sql = """
with lead_data as 
(
    select 
    channel
    , max(lead_created_at) as max_dt
    , min(lead_created_at) as min_dt
    , count(distinct candidate_id) as leads_count
    from 
    (
        select 
        candidate_id 
        , strftime('%Y-%m-%d', lead_created_at) as lead_created_at
        , channel
        from leads
    )
    group by channel
)

select * 
from lead_data 
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,channel,max_dt,min_dt,leads_count
0,Facebook,2019-02-27,2019-01-01,1523
1,SEM_Bing,2019-02-27,2019-01-01,243
2,SEM_Google,2019-02-27,2019-01-01,7296


In [15]:
sql = """
with marketing_data as 
(
    select 
    count(*)
    , count(spend_bing) as not_null_spend_bing
    , count(spend_facebook) as not_null_spend_facebook
    , count(spend_adwords) as not_null_spend_adwords
    , count(case when spend_bing = 0 then null else spend_bing end) as not_zero_spend_bing
    , count(case when spend_facebook = 0 then null else spend_facebook end) as not_zero_spend_facebook
    , count(case when spend_adwords = 0 then null else spend_adwords end) as not_zero_spend_adwords
    , max(date) as max_date
    , min(date) as min_date
    from 
    (
        select  
        strftime('%Y-%m-%d', date) as date
        , spend_bing
        , spend_facebook
        , spend_adwords
        from marketing 
    )
)

select * 
from marketing_data 
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res


,count(*),not_null_spend_bing,not_null_spend_facebook,not_null_spend_adwords,not_zero_spend_bing,not_zero_spend_facebook,not_zero_spend_adwords,max_date,min_date
0,59,59,59,59,59,59,59,2019-02-28,2019-01-01


That means we can use marketing and leads data almost without any additional conditions related to nullable values or different time periods. 

The only condition - I'm going to exclude date 2019-02-28. We use only 2019-01-01 - 2019-02-27 interval for leads so it's no matter which spends were before of after this interval. Let's use following condition for further selects related to marketing table.

In [16]:
sql = """
with date_limits as 
(
    select 
    max(lead_created_at) as max_dt
    , min(lead_created_at) as min_dt
    from 
    (
        select 
        strftime('%Y-%m-%d', lead_created_at) as lead_created_at
        from leads
    )
)

select count(*) 
from marketing
where strftime('%Y-%m-%d', date) >= (select min_dt from date_limits)
and strftime('%Y-%m-%d', date) <= (select max_dt from date_limits)
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,count(*)
0,58


### Validation point 3. 

Also let's check if we have some deals that have been created before leads were created.

In [17]:
sql = """
select 
d.deal_id as deal_id
, d.lead_id as lead_id
, strftime('%Y-%m-%d', d.deal_created_at) as deal_created_at
, strftime('%Y-%m-%d', l.lead_created_at) as lead_created_at
from deals as d
inner join leads as l 
on d.lead_id = l.lead_id
where deal_created_at <= lead_created_at
limit 100
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,deal_id,lead_id,deal_created_at,lead_created_at
0,58761,60327,2019-01-25,2019-02-01
1,32802,53958,2018-08-13,2019-01-02
2,51475,56764,2018-12-11,2019-01-16
3,61413,62664,2019-02-07,2019-02-13
4,35045,53958,2018-08-29,2019-01-02
5,59992,60327,2019-01-31,2019-02-01
6,58033,60969,2019-01-22,2019-02-05
7,49524,53958,2018-11-29,2019-01-02
8,37238,53958,2018-09-13,2019-01-02
9,50673,53958,2018-12-06,2019-01-02


I'm not sure what is the exactly reason for such cases existing but further I'm going to exclude all of these lead_id and deal_id as some error cases.

# The Solution

##  1. Most profitable marketing channel.

Let's compare the overall number of leads and cost per each lead by channels.

In [18]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
),

lead_data as 
(
    select 
    channel
    , count(distinct candidate_id) as leads_count
    from 
    (
        select 
        candidate_id
        , channel
        from leads as l 
        inner join leads_unique as lu 
        on l.lead_id = lu.lead_id
        left join exclusions as e
        on l.lead_id = e.lead_id 
        where e.lead_id is null
    )
    group by channel
),

date_limits as 
(
    select 
    max(lead_created_at) as max_dt
    , min(lead_created_at) as min_dt
    from 
    (
        select 
        strftime('%Y-%m-%d', lead_created_at) as lead_created_at
        from leads
    )
), 

marketing_sum as
(
    select
    count(*) as marketing_days
    , sum(spend_bing) as spend_bing
    , sum(spend_facebook) as spend_facebook
    , sum(spend_adwords) as spend_adwords
    from marketing 
    where strftime('%Y-%m-%d', date) >= (select min_dt from date_limits)
    and strftime('%Y-%m-%d', date) <= (select max_dt from date_limits)    
)

select 
channel
, cast(round(total) as int) as total_spends
, leads_count
, round((total / leads_count), 2) as spends_per_lead
from 
(
    select 
    t.*
    , leads_count
    from 
    (
        select 
        "Facebook" as channel
        , spend_facebook as total 
        from marketing_sum 

        union

        select 
        "SEM_Bing" as channel
        , spend_bing
        from marketing_sum 

        union

        select
        "SEM_Google" as channel
        , spend_adwords
        from marketing_sum 
    ) as t 
    inner join lead_data as l 
    on t.channel = l.channel
)    
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,channel,total_spends,leads_count,spends_per_lead
0,Facebook,158726,1520,104.43
1,SEM_Bing,52225,243,214.92
2,SEM_Google,320562,7294,43.95


### 1. Result

SEM_Google is the most profitable according to the cheapest leads: 44 per one lead as well as SEM_Bing is the most expensive channel with the result of 215 per one lead.

##  2. Problematic recruitment agent(s)

One lead could be involved to several deals and I'm sure it's no matter for us how many deals per same lead is available. So we're going to calculate not the rate of won deals among all deals but the rate of converted to employees leads. It means that in case when at least one won deal exists per lead we consider it as successful case (no matter how many other lost deals per this lead we have). 

In [19]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
)

        select count(distinct d.lead_id)
        , count(distinct d.deal_id)
        from deals as d
        inner join leads_unique as lu 
        on d.lead_id = lu.lead_id        
        left join exclusions as e
        on d.lead_id = e.lead_id 
        where e.lead_id is null
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,count(distinct d.lead_id),count(distinct d.deal_id)
0,491,961


In [20]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
)

        select count(distinct d.lead_id)
        , count(distinct d.deal_id)
        from deals as d
        inner join leads_unique as lu 
        on d.lead_id = lu.lead_id        
        left join exclusions as e
        on d.lead_id = e.lead_id 
        where e.lead_id is null
        and status = "won"
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,count(distinct d.lead_id),count(distinct d.deal_id)
0,93,94


Well, 491 leads were involved to 961 deals. 93 of leads became employees and let's consider that it could be only once per all timeperiod.

Let's check how many agents could be assigned to the same lead.

In [21]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
)

select 
agent_id
, lead_id
, rnk
, max(rnk) over (partition by lead_id) as max_rnk
from 
(
    select 
    agent_id
    , lead_id
    , row_number() over (partition by lead_id) as rnk
    from 
    (
        select distinct
        agent_id
        , d.lead_id as lead_id
        from deals as d
        inner join leads_unique as lu 
        on d.lead_id = lu.lead_id        
        left join exclusions as e
        on d.lead_id = e.lead_id 
        where e.lead_id is null
    )
)
order by max_rnk desc
limit 6
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,agent_id,lead_id,rnk,max_rnk
0,3,56591,1,3
1,4,56591,2,3
2,5,56591,3,3
3,2,64157,1,3
4,4,64157,2,3
5,5,64157,3,3


Above is the example of 3 different agents per the same lead, so how often are such cases?

In [22]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
)


select count(distinct lead_id) 
from 
(
    select 
    agent_id
    , lead_id
    , rnk
    , max(rnk) over (partition by lead_id) as max_rnk
    from 
    (
        select 
        agent_id
        , lead_id
        , row_number() over (partition by lead_id) as rnk
        from 
        (
            select distinct
            agent_id
            , d.lead_id as lead_id
            from deals as d
            inner join leads_unique as lu 
            on d.lead_id = lu.lead_id        
            left join exclusions as e
            on d.lead_id = e.lead_id 
            where e.lead_id is null
        )
    )
)
where max_rnk > 1
limit 6
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,count(distinct lead_id)
0,26


Well, 26 leads had two or more agents. 

Let's decide that 1 unique lead for the agent is an attempt and let's calculate how many of them were successful finally. In case when the same lead had two or more agents, the success would be allocated to those agent who get there first.

Also here we're going to ignore 'opened' deals. And also ignore such leads who have this 'open' status regardless existing 'lost'. By the way, is it possible that the same lead is opened and won at the same time? 

In [23]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
),

opened_leads as
(
    select distinct 
    d.lead_id as lead_id 
    , d.status as status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    where e.lead_id is null
    and d.status = 'open'
)

    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    , d.status as status
    , ol.status as opened_status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    inner join opened_leads as ol
    on d.lead_id = ol.lead_id
    where e.lead_id is null
    and d.status = 'won'
order by lead_id desc
limit 5
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,deal_id,lead_id,status,opened_status
0,64322,62239,won,open
1,61856,56757,won,open


In [24]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
),

opened_leads as
(
    select distinct 
    d.lead_id as lead_id 
    , d.status as status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    where e.lead_id is null
    and d.status = 'open'
)

select 
count(distinct deal_id) as deals
, count(distinct lead_id) as leads
from
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    , d.status as status
    , ol.status as opened_status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    inner join opened_leads as ol
    on d.lead_id = ol.lead_id
    where e.lead_id is null
    and d.status = 'lost'
)
limit 5
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,deals,leads
0,56,25


Well, let's exclude those 25 leads which are opened and also have already had 'lost' status because they still have the possibility to success.
At the same way 'won' and 'open' leads will participate in following calculation but 'open' rows will be excluded.

In [25]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
),

opened_leads as
(
    select distinct 
    d.lead_id as lead_id 
    , d.status as status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    where e.lead_id is null
    and d.status = 'open'
), 

lost_leads as
(
    select distinct 
    d.lead_id as lead_id 
    , d.status as status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    where e.lead_id is null
    and d.status = 'lost'
),

opened_leads_exclusion as
(
    select distinct
    ll.lead_id as lead_id
    from lost_leads as ll
    inner join opened_leads as ol
    on ol.lead_id = ll.lead_id
)

select 
agent_id
, count(distinct deal_id) as deals_closed
, count(distinct deal_won) as deals_won
, round((count(distinct deal_won) / cast(count(distinct deal_id) as float)), 4) as won_deals_rate

, count(distinct lead_id) as leads_closed
, count(distinct lead_won) as leads_won
, round((count(distinct lead_won) / cast(count(distinct lead_id) as float)), 4) as leads_won_rate

, cast(sum(revenue) as int) as total_revenue
, round((sum(revenue) / cast(count(distinct lead_id) as float)), 1) as revenue_per_lead
, round((sum(revenue) / cast(count(distinct lead_won) as float)), 1) as revenue_per_win 

from 
(
    select 
    agent_id
    , deal_id
    , deal_won
    , revenue
    , lead_id
    , lead_won
    from
    (
        select 
        d.agent_id as agent_id
        , d.lead_id as lead_id
        , d.deal_id as deal_id 
        , (case when d.status = "won" then d.deal_id else null end) as deal_won        
        , (case when d.status = "won" then d.lead_id else null end) as lead_won 
        , (case when d.status = "won" then d.value else null end) as revenue
        from deals as d
        inner join leads_unique as lu 
        on d.lead_id = lu.lead_id        
        left join exclusions as e
        on d.lead_id = e.lead_id 
        left join opened_leads_exclusion as ole
        on d.lead_id = ole.lead_id
        where e.lead_id is null
        and ole.lead_id is null 
        and d.status in ('won', 'lost')
    )
)
group by agent_id
limit 20
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,agent_id,deals_closed,deals_won,won_deals_rate,leads_closed,leads_won,leads_won_rate,total_revenue,revenue_per_lead,revenue_per_win
0,1,100,15,0.1500,73,15,0.2055,97500,1335.6,6500.0
1,2,222,30,0.1351,118,29,0.2458,221500,1877.1,7637.9
2,3,307,34,0.1107,172,34,0.1977,264200,1536.0,7770.6
3,4,46,1,0.0217,35,1,0.0286,15000,428.6,15000.0
4,5,179,12,0.0670,74,12,0.1622,87000,1175.7,7250.0


### 2. Result

Well, we can notice that agent 2 performs better than anyone with 24.58% success rate of leads conversation to won leads. This agent also shows the best ratio of revenue per 1 lead with whom the agent started to work. 

I guess we need to exclude from results the agent 4 due to small overall number of leads and deals started and conclude that currently agent 5 is the worst among all 4 agents.

Basically I guess we don't need to evaluate the number of deals and successful deals per agent because the final won result per lead is the only that has matter in this case.

##  3. Company wide CLTV divided by CAC.

Consider CAC as total revenue divided by total customers number (leads in our case).
Consider LTV as all revenue per period as well as CLTV as all revenue divided by total customers number (leads).

In [26]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
),

leads_count as 
(
    select 
    'all period' as period
    , count(distinct l.lead_id) as leads_count
    from leads as l 
    inner join leads_unique as lu 
    on l.lead_id = lu.lead_id
    left join exclusions as e
    on l.lead_id = e.lead_id 
    where e.lead_id is null
),

date_limits as 
(
    select 
    max(lead_created_at) as max_dt
    , min(lead_created_at) as min_dt
    from 
    (
        select 
        strftime('%Y-%m-%d', lead_created_at) as lead_created_at
        from leads
    )
), 

marketing_sum as
(
    select
    'all period' as period
    , sum(spend_bing + spend_facebook + spend_adwords) as marketing_sum
    from marketing 
    where strftime('%Y-%m-%d', date) >= (select min_dt from date_limits)
    and strftime('%Y-%m-%d', date) <= (select max_dt from date_limits)    
), 

total_revenue as 
(
    select 
    'all period' as period
    , sum(value) as total_revenue
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    where e.lead_id is null
    and d.status = 'won'
)

select 
cast(round(marketing_sum) as int) as marketing_sum 
, cast(round(total_revenue) as int) as total_revenue  
, leads_count
, round((total_revenue / leads_count),2) as CLTV
, round((marketing_sum / leads_count),2) as CAC
, round((total_revenue / marketing_sum),2) as CLTV_div_by_CAC
from marketing_sum as m
inner join total_revenue as r 
on m.period = r.period
inner join leads_count as l
on m.period = l.period


limit 20
"""
con = sl.connect("mwdwing_base.db")
pd.read_sql_query(sql, con)

,marketing_sum,total_revenue,leads_count,CLTV,CAC,CLTV_div_by_CAC
0,531513,705200,9057,77.86,58.69,1.33


### 3. Answer

The company CLTV divided by CAC: 1.33

###  4. Approaches to determine varying levels of conservativeness for the previous answer (how optimistic or pessimistic calculation is).

We calculate above something like fact CLTV, at the same time, for sure, the timedelta between lead and deal created in case of success is distributed somehow.

In [27]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
)

select 
min_date_diff
, min(per_20) as per_20
, min(per_50) as per_50
, min(per_80) as per_80
, max_date_diff
from
(
    select
    (case when window_percentage > 0.2 then date_diff else null end ) as per_20
    , (case when window_percentage > 0.5 then date_diff else null end ) as per_50
    , (case when window_percentage > 0.8 then date_diff else null end ) as per_80
    , max(date_diff) over () as max_date_diff
    , min(date_diff) over () as min_date_diff
    from 
    (
        select 
        date_diff
        , frequence
        , percentage
        , sum(percentage) over (order by date_diff) as window_percentage
        from 
        (
            select 
            date_diff
            , count(*) as frequence
            , (count(*) / cast(cnt as float)) as percentage
            from 
            (
                select 
                d.lead_id as lead_id
                , count(*) over () as cnt 
                , cast((
                julianday(strftime('%Y-%m-%d', d.deal_created_at))
                - 
                julianday(strftime('%Y-%m-%d', l.lead_created_at))
                ) as int) as date_diff
                from deals as d 
                inner join leads as l 
                on d.lead_id = l.lead_id
                left join exclusions as e
                on l.lead_id = e.lead_id 
                where e.lead_id is null
                and d.status = "won"
            )
            group by date_diff
        )
    )
)
group by 
min_date_diff
, max_date_diff
limit 100
"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,min_date_diff,per_20,per_50,per_80,max_date_diff
0,1,6,15,32,107


We can notice that in case of won deals the median timedelta between lead and deal created is 15 days as well as for 80% of successful deals this delta is 32 days. 

Basically for LTV we can calculate something like Lifetime * ARPU and Lifetime could be measured as an integral function of retention from time. 

But in our case customer make only one successful deal. So I don't think we need to concetrate on some values like retention, churn, etc. because only one successful deal per lead is available.

Consider that we'll have the same budget for marketing. Different proportions of spends per channel will bring us different number of leads. 

We've already evaluated it per channel, so let's consider that the best variant is when all leads are from SEM_Google - that means that SEM_Google has unlimited number of potential leads. 

At the same time the worst is related to such scenario when all cheap leads from SEM_Google were ended and we have to divide our spends between Facebook and SEM_Bing with the same proportion as it was previously.

After that consider that proportion of leads converted to deals is the same.

Ratio of won leads in best variant should be at currently maximum level (i.e. agent 2 result from previous case) as well as the worst scenario corresponds to the result of agent 5.

Finally the comission from startups also could be different. Let's take it also from agent's results.

In [28]:
sql = """
with exclusions as
(
    select 
    d.deal_id as deal_id
    , d.lead_id as lead_id
    from deals as d
    inner join leads as l 
    on d.lead_id = l.lead_id
    where deal_created_at <= lead_created_at
),

leads_unique as
(
    select 
    lead_id
    from
    (
        select 
        lead_id
        , row_number() over (partition by candidate_id order by lead_created_at) as rnk 
        from leads 
    )
    where rnk = 1
),

lead_data as 
(
    select 
    channel
    , count(distinct candidate_id) as leads_count
    from 
    (
        select 
        candidate_id
        , channel
        from leads as l 
        inner join leads_unique as lu 
        on l.lead_id = lu.lead_id
        left join exclusions as e
        on l.lead_id = e.lead_id 
        where e.lead_id is null
    )
    group by channel
),

date_limits as 
(
    select 
    max(lead_created_at) as max_dt
    , min(lead_created_at) as min_dt
    from 
    (
        select 
        strftime('%Y-%m-%d', lead_created_at) as lead_created_at
        from leads
    )
), 

marketing_sum as
(
    select
    count(*) as marketing_days
    , sum(spend_bing) as spend_bing
    , sum(spend_facebook) as spend_facebook
    , sum(spend_adwords) as spend_adwords
    from marketing 
    where strftime('%Y-%m-%d', date) >= (select min_dt from date_limits)
    and strftime('%Y-%m-%d', date) <= (select max_dt from date_limits)    
),

lead_to_deal_conversion as 
(    
    select 
    'all period' as period
    , round
    (
        (count(distinct d.lead_id) 
        /
        cast(count(distinct l.lead_id) as float) )
    , 5) as lead_to_deal_conversion    
    from leads as l 
    left join deals as d
    on l.lead_id = d.lead_id
    inner join leads_unique as lu 
    on l.lead_id = lu.lead_id
    left join exclusions as e
    on l.lead_id = e.lead_id 
    where e.lead_id is null
),

opened_leads as
(
    select distinct 
    d.lead_id as lead_id 
    , d.status as status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    where e.lead_id is null
    and d.status = 'open'
), 

lost_leads as
(
    select distinct 
    d.lead_id as lead_id 
    , d.status as status
    from deals as d
    inner join leads_unique as lu 
    on d.lead_id = lu.lead_id        
    left join exclusions as e
    on d.lead_id = e.lead_id 
    where e.lead_id is null
    and d.status = 'lost'
),

opened_leads_exclusion as
(
    select distinct
    ll.lead_id as lead_id
    from lost_leads as ll
    inner join opened_leads as ol
    on ol.lead_id = ll.lead_id
), 

leads_won_rate as 
(
    select 
    'all period' as period
    , max(leads_won_rate) as max_leads_won_rate
    , min(leads_won_rate) as min_leads_won_rate
    , max(revenue_per_win) as max_revenue_per_win
    , min(revenue_per_win) as min_revenue_per_win
    from 
    (
        select 
        agent_id
        , round((count(distinct lead_won) / cast(count(distinct lead_id) as float)), 4) as leads_won_rate
        , round((sum(revenue) / cast(count(distinct lead_won) as float)), 1) as revenue_per_win 
        from 
        (
            select 
            agent_id
            , deal_id
            , deal_won
            , revenue
            , lead_id
            , lead_won
            from
            (
                select 
                d.agent_id as agent_id
                , d.lead_id as lead_id
                , d.deal_id as deal_id 
                , (case when d.status = "won" then d.deal_id else null end) as deal_won        
                , (case when d.status = "won" then d.lead_id else null end) as lead_won 
                , (case when d.status = "won" then d.value else null end) as revenue
                from deals as d
                inner join leads_unique as lu 
                on d.lead_id = lu.lead_id        
                left join exclusions as e
                on d.lead_id = e.lead_id 
                left join opened_leads_exclusion as ole
                on d.lead_id = ole.lead_id
                where e.lead_id is null
                and ole.lead_id is null 
                and d.status in ('won', 'lost')
            )
        )
        group by agent_id
    )
    where agent_id in (2, 5)
),

max_leads_won_rate as 
(
    select
    'best scenario' as variant
    , max_leads_won_rate as leads_won_rate
    , max_revenue_per_win as revenue_per_win
    from leads_won_rate
),

min_leads_won_rate as 
(
    select
    'worst scenario' as variant
    , min_leads_won_rate as leads_won_rate
    , min_revenue_per_win as revenue_per_win
    from leads_won_rate
),

cac_per_channels as 
(
    select 
    'all period' as period
    , channel
    , cast(round(total) as int) as total_spends
    , leads_count
    , round((total / leads_count), 2) as spends_per_lead
    from 
    (
        select 
        t.*
        , leads_count
        from 
        (
            select 
            "Facebook" as channel
            , spend_facebook as total 
            from marketing_sum 

            union

            select 
            "SEM_Bing" as channel
            , spend_bing
            from marketing_sum 

            union

            select
            "SEM_Google" as channel
            , spend_adwords
            from marketing_sum 
        ) as t 
        inner join lead_data as l 
        on t.channel = l.channel
    )    
),

marketing_sum_overall as
(
    select
    'all period' as period
    , sum(spend_bing + spend_facebook + spend_adwords) as marketing_sum
    from marketing 
    where strftime('%Y-%m-%d', date) >= (select min_dt from date_limits)
    and strftime('%Y-%m-%d', date) <= (select max_dt from date_limits)    
)

select 
t.*
, ldc.lead_to_deal_conversion as lead_to_deal_conversion
, cast( round (ldc.lead_to_deal_conversion * t.leads_count) as int) as deal_leads_opened
, coalesce(ma.leads_won_rate , mi.leads_won_rate) as leads_won_rate
, cast(round ((ldc.lead_to_deal_conversion * t.leads_count) * coalesce(ma.leads_won_rate , mi.leads_won_rate)) as int) as won_leads
, coalesce(ma.revenue_per_win , mi.revenue_per_win) as revenue_per_win
, ((coalesce(ma.revenue_per_win , mi.revenue_per_win) * 
(ldc.lead_to_deal_conversion * t.leads_count) * coalesce(ma.leads_won_rate , mi.leads_won_rate))
/ t.leads_count) as CLTV
, (((coalesce(ma.revenue_per_win , mi.revenue_per_win) * 
(ldc.lead_to_deal_conversion * t.leads_count) * coalesce(ma.leads_won_rate , mi.leads_won_rate))
/ t.leads_count) / CAC) as CLT_div_by_CAC

from 
(
    select 
    'all period' as period
    , 'best scenario' as variant
    , round(marketing_sum) as marketing_sum
    , spends_per_lead as CAC
    , cast(round((marketing_sum / spends_per_lead)) as int) as leads_count 
    from marketing_sum_overall as m
    inner join cac_per_channels as c 
    on m.period = c.period 
    where c.channel = "SEM_Google"

    union all

    select 
    'all period' as period
    , 'worst scenario' as variant
    , round(marketing_sum) as marketing_sum
    , round(sum(spends_per_lead),2) as CAC
    , cast(round(sum(leads_count)) as int) as leads_count
    from 
    (
        select 
        t.channel
        , m.marketing_sum as marketing_sum
        , (t.spends_per_lead * t.overall_spends) as spends_per_lead 
        , (t.overall_spends * m.marketing_sum) / t.spends_per_lead as leads_count
        from marketing_sum_overall as m
        inner join  
        (
            select 
            'all period' as period
            , channel
            , total_spends / cast( (sum(total_spends) over () ) as float)  as overall_spends
            , spends_per_lead
            from cac_per_channels
            where channel != "SEM_Google"
        ) as t 
        on t.period = m.period
    )
    group by marketing_sum
) as t
left join lead_to_deal_conversion as ldc
on t.period = ldc.period
left join max_leads_won_rate as ma
on t.variant = ma.variant
left join min_leads_won_rate as mi 
on t.variant = mi.variant

"""
con = sl.connect("mwdwing_base.db")
res = pd.read_sql_query(sql, con)
res

,period,variant,marketing_sum,CAC,leads_count,lead_to_deal_conversion,deal_leads_opened,leads_won_rate,won_leads,revenue_per_win,CLTV,CLT_div_by_CAC
0,all period,best scenario,531513.0,43.95,12094,0.05421,656,0.2458,161,7637.9,101.773627,2.315668
1,all period,worst scenario,531513.0,131.78,4442,0.05421,241,0.1622,39,7250.0,63.748250,0.483748


### 4. Answer 

- Max expected CLTV / CAC: 2.32
- Min expected CLTV / CAC: 0.48